<a href="https://colab.research.google.com/github/harjeet88/llm-course/blob/main/4_module/1_simple_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Step 1: Install the Google Generative AI SDK
!pip install -q openai google-generativeai

In [26]:
# Step 2: Import libraries
import os
import openai
from openai import OpenAI
from google.colab import userdata
#import google.generativeai as genai
from IPython.display import display, Markdown

In [32]:
# Step 3
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY=GOOGLE_API_KEY
gemini_model='gemini-2.5-flash'
gemini_api_endpoint="https://generativelanguage.googleapis.com/v1beta/openai/"

In [84]:
# Step 4: Initialize the model (using Gemini flash)
# Configure OpenAI client to use Google's Gemini endpoint
gemini = OpenAI(api_key=GOOGLE_API_KEY, base_url=gemini_api_endpoint)

In [83]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from huggingface_hub import InferenceClient
import os

In [86]:
# Set Hugging Face API token (get a free token from huggingface.co)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACE_API_KEY')

In [90]:
!echo "i am harjeet. i work at bla bla company. i am from Bangalore India." > "sample_ai_textbook.txt"

In [91]:
# Step 1: Load and chunk document
with open("sample_ai_textbook.txt", "r") as file:
    text = file.read()

In [92]:
# Simple chunking by splitting into paragraphs (approx. 500 chars)
chunks = [text[i:i+500] for i in range(0, len(text), 500)]
print(f"Created {len(chunks)} chunks")

Created 1 chunks


In [93]:
# Step 2: Create embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(chunks, convert_to_numpy=True)
dimension = embeddings.shape[1]  # Embedding size (e.g., 384)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [94]:
# Step 3: Store embeddings in FAISS
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print("Embeddings stored in FAISS")

Embeddings stored in FAISS


In [98]:
def retrive_chunk(question):
  question_embedding = model.encode([question])[0]
  D, I = index.search(np.array([question_embedding]), k=1)  # Find top 1 chunk
  retrieved_chunk = chunks[I[0][0]]
  print("Retrieved chunk:", retrieved_chunk[:100], "...")
  return retrieved_chunk

In [111]:
def getAnswer(question) :
  retrieved_chunk = retrive_chunk(question)
  system="you are a LLM using RAG to answer user questions. you are impersonating the person mentioned in RAG. so strictly build answers based on context. If you donot know answer, say i dont know  "
  prompt = f"Context: {retrieved_chunk}\n\nQuestion: {question}\nAnswer:"
  answer = gemini.chat.completions.create(
      model=gemini_model,
      messages=[
          {"role": "system", "content": system},
          {"role": "user", "content": prompt}
      ]
  ).choices[0].message.content
  return answer

In [114]:
# Step 4: Query the system
question = "where do you work?"
answer=getAnswer(question)
print("Question:", question)
print("Answer:", answer)

Retrieved chunk: i am harjeet. i work at bla bla company. i am from Bangalore India.
 ...
Question: where do you work?
Answer: I work at bla bla company.
